<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [14]</a>'.</span>

# Initial Setups

## (Google Colab use only)

In [1]:
# Use Google Colab
use_colab = False

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/cs696ds_lexalytics/Ronald Gypsum Prompts'
    
    # Install packages specified in requirements
    !pip install -r requirements.txt

    %cd 'prompts_subtask4'
    
    # List the directory contents
    !ls

## Experiment parameters

In [2]:
# We will use the following string ID to identify this particular (training) experiments
# in directory paths and other settings
experiment_id = 'bert_16_shot_prompt_logit_softmax_atsc_restaurants_bert_yelp_restaurants_multiple_prompts_589'

# Random seed
random_seed = 589

# path to pretrained MLM model folder or the string "bert-base-uncased"
lm_model_path = 'bert-base-uncased'

# Prompts to be added to the end of each review text
# Note: pseudo-labels for each prompt should be given in the order of (positive), (negative), (neutral)
sentiment_prompts = [
    {"prompt": "I felt the {aspect} was [MASK].", "labels": ["good", "bad", "ok"]},
    {"prompt": "I [MASK] the {aspect}.", "labels": ["love", "hate", "dislike"]},
    {"prompt": "The {aspect} made me feel [MASK].", "labels": ["good", "bad", "indifferent"]},
    {"prompt": "The {aspect} is [MASK].", "labels": ["good", "bad", "ok"]}
]

# Multiple prompt merging behavior
prompts_merge_behavior = 'sum_logits'

# Perturb the input embeddings of tokens within the prompts
prompts_perturb = False

# Test settings
testing_batch_size = 32
testing_domain = 'restaurants' # 'laptops', 'restaurants', 'joint'

In [3]:
# Parameters
experiment_id = "bert_1024_shot_prompt_logit_softmax_atsc_restaurants_bert_yelp_restaurants_single_prompt_i_felt_696"
random_seed = 696
lm_model_path = "../trained_models/lm_further_pretraining_bert_yelp_restaurants_bseoh_2021-03-22--15_03_31"
sentiment_prompts = [
    {"prompt": "I felt the {aspect} was [MASK].", "labels": ["good", "bad", "ok"]}
]
testing_batch_size = 32
testing_domain = "restaurants"
prompts_merge_behavior = "sum_logits"
prompts_perturb = False


In [4]:
# Batch size adjustment for multiple prompts.
testing_batch_size = testing_batch_size // len(sentiment_prompts)

In [5]:
print("Experiment ID:", experiment_id)

Experiment ID: bert_1024_shot_prompt_logit_softmax_atsc_restaurants_bert_yelp_restaurants_single_prompt_i_felt_696


## Package imports

In [6]:
import sys
import os
import random
import shutil
import copy
import inspect
import json

import numpy as np
import torch
import transformers
import datasets
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tqdm

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import utils

# Random seed settings
random.seed(random_seed)
np.random.seed(random_seed)

# cuBLAS reproducibility
# https://docs.nvidia.com/cuda/cublas/index.html#cublasApi_reproducibility
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"
torch.set_deterministic(True)
torch.manual_seed(random_seed)

# Print version information
print("Python version: " + sys.version)
print("NumPy version: " + np.__version__)
print("PyTorch version: " + torch.__version__)
print("Transformers version: " + transformers.__version__)

Intel(R) oneAPI Data Analytics Library solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html


Python version: 3.7.9 (default, Nov  4 2020, 21:30:07) 
[GCC 7.3.0]
NumPy version: 1.19.2
PyTorch version: 1.7.1
Transformers version: 4.3.3


## PyTorch GPU settings

In [7]:
if torch.cuda.is_available():    
    torch_device = torch.device('cuda')

    # Set this to True to make your output immediately reproducible
    # Note: https://pytorch.org/docs/stable/notes/randomness.html
    torch.backends.cudnn.deterministic = True
    
    # Disable 'benchmark' mode: Set this False if you want to measure running times more fairly
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = False
    
    # Faster Host to GPU copies with page-locked memory
    use_pin_memory = True
    
    # Number of compute devices to be used for training
    training_device_count = torch.cuda.device_count()

    # CUDA libraries version information
    print("CUDA Version: " + str(torch.version.cuda))
    print("cuDNN Version: " + str(torch.backends.cudnn.version()))
    print("CUDA Device Name: " + str(torch.cuda.get_device_name()))
    print("CUDA Capabilities: "+ str(torch.cuda.get_device_capability()))
    print("Number of CUDA devices: "+ str(training_device_count))
    
else:
    torch_device = torch.device('cpu')
    use_pin_memory = False
    
    # Number of compute devices to be used for training
    training_device_count = 1

print()
print("PyTorch device selected:", torch_device)

CUDA Version: 11.0
cuDNN Version: 8005
CUDA Device Name: GeForce GTX 1080 Ti
CUDA Capabilities: (6, 1)
Number of CUDA devices: 1

PyTorch device selected: cuda


# Prepare Datasets for Prompt-based Classifier

## Load the SemEval dataset

In [8]:
# Load semeval for both domains
restaurants_dataset = datasets.load_dataset(
    os.path.abspath('../dataset_scripts/semeval2014_task4/semeval2014_task4.py'),
    name="SemEval2014Task4Dataset - Subtask 4",
    data_files={
        'test': '../dataset_files/semeval_2014/Restaurants_Test_Gold.xml',
        'train': '../dataset_files/semeval_2014/Restaurants_Train_v2.xml',
    },
    cache_dir='../dataset_cache')

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)


In [9]:
# The dataset chosen for testing
if testing_domain == 'restaurants':
    test_set = restaurants_dataset['test']

In [10]:
print(len(test_set))

973


In [11]:
print(test_set[4])

{'aspect': 'ambience', 'sentiment': 0, 'text': 'Certainly not the best sushi in New York, however, it is always fresh, and the place is very clean, sterile.'}


# Zero-shot ATSC with Prompts

## Initialize BERT MLM

In [12]:
# Load pretrained language model
# Use the config, but not the actual weights, from the pretrained model
lm_config = transformers.AutoConfig.from_pretrained('bert-base-uncased', cache_dir='../bert_base_cache')
lm = transformers.AutoModelForMaskedLM.from_config(lm_config)
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir='../bert_base_cache')

## Define a new model with non-trainable softmax head

In [13]:
# Encode the pseudo-label words for each sentiment class
sentiment_word_ids = []

for sp in sentiment_prompts:
    sentiment_word_ids.append(
        [tokenizer.convert_tokens_to_ids(w) for w in sp['labels']])

print(sentiment_word_ids)

classifier_model = utils.MultiPromptLogitSentimentClassificationHead(
    lm=lm,
    num_class=3,
    num_prompts=len(sentiment_prompts), pseudo_label_words=sentiment_word_ids,
    target_token_id=tokenizer.mask_token_id,
    merge_behavior=prompts_merge_behavior,
    perturb_prompts=prompts_perturb)

classifier_model = classifier_model.to(device=torch_device)

[[2204, 2919, 7929]]
Detected LM type: bert


## Load our saved weights

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [14]:
# Locate the weight file.
trained_model_directory = os.path.join('..', 'trained_models_prompts', experiment_id)

saved_weights_name = ''

for fname in os.listdir(trained_model_directory):
    if fname.startswith('epoch'):
        saved_weights_name = fname
        break

print("Loading", saved_weights_name)

classifier_model.load_state_dict(torch.load(
    os.path.join(trained_model_directory, saved_weights_name),
    map_location=torch_device))

FileNotFoundError: [Errno 2] No such file or directory: '../trained_models_prompts/bert_1024_shot_prompt_logit_softmax_atsc_restaurants_bert_yelp_restaurants_single_prompt_i_felt_696'

## Evaluation with in-domain test set



In [ ]:
def compute_metrics(predictions, labels):
    preds = predictions.argmax(-1)

    precision, recall, f1, _ = sklearn.metrics.precision_recall_fscore_support(
        y_true=labels, y_pred=preds, labels=[0,1,2], average='macro')

    acc = sklearn.metrics.accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
test_dataloader = torch.utils.data.DataLoader(
    test_set, batch_size=testing_batch_size, pin_memory=use_pin_memory)

In [ ]:
with torch.no_grad():
    classifier_model.eval()

    predictions_test = torch.Tensor([])
    labels_test = torch.Tensor([])

    for batch_val in tqdm.notebook.tqdm(test_dataloader):

        reviews_repeated = []
        prompts_populated = []

        for prompt in sentiment_prompts:
            reviews_repeated = reviews_repeated + batch_val["text"]

            for aspect in batch_val["aspect"]:
                prompts_populated.append(prompt['prompt'].format(aspect=aspect))

        batch_encoded = tokenizer(
            reviews_repeated, prompts_populated,
            padding='max_length', truncation='only_first', max_length=256,
            return_tensors='pt')
        
        batch_encoded.to(torch_device)

        labels = batch_val["sentiment"]

        outputs = classifier_model(batch_encoded)

        outputs = outputs.to('cpu')

        predictions_test = torch.cat([predictions_test, outputs])
        labels_test = torch.cat([labels_test, labels])

    # Compute metrics
    test_metrics = compute_metrics(predictions_test, labels_test)

    print(test_metrics)

    # Save test_metrics into a file for later processing
    with open(os.path.join(trained_model_directory, 'test_metrics_subtask4.json'), 'w') as test_metrics_json:
        json.dump(test_metrics, test_metrics_json)

## Results visualization

In [ ]:
# Calculate metrics and confusion matrix based upon predictions and true labels
cm = sklearn.metrics.confusion_matrix(labels_test.detach().numpy(), predictions_test.detach().numpy().argmax(-1))

df_cm = pd.DataFrame(
    cm,
    index=[i for i in ["positive", "negative", "neutral"]],
    columns=[i for i in ["positive", "negative", "neutral"]])

plt.figure(figsize=(10, 7))

ax = sn.heatmap(df_cm, annot=True)

ax.set(xlabel='Predicted Label', ylabel='True Label')
plt.show()